### DSC 4320 Final Project
Brock Carey, Emily Liau

**Environment Set-Up**

In [33]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import math

**Data Pre-Processing**

In [3]:
pwd

'C:\\Users\\bacar\\4320\\Final Project\\callpredict'

In [5]:
# import data into environment
data = pd.read_csv('C:/Users/bacar/4320/Final Project/data.csv', sep=',')
data

,DATE_FOR,RTD_ST_CD,CustomerSegment,Tenure,Age,MART_STATUS,GENDER,CHANNEL1_6M,CHANNEL2_6M,CHANNEL3_6M,...,CHANNEL5_3M,METHOD1_3M,PAYMENTS_3M,NOT_DI_3M,NOT_DI_6M,EVENT1_30_FLAG,EVENT2_90_SUM,LOGINS,POLICYPURCHASECHANNEL,Call_Flag
0,5/19/2014,ST_S0,1,16.175222,78.403833,MS_S0,F,0.0,0.0,1.0,...,2,0,3,0,0,0,0,0,0,0
1,5/17/2014,ST_S0,1,15.931554,70.989733,MS_S1,F,0.0,6.0,0.0,...,0,3,3,0,0,0,0,0,0,0
2,5/15/2014,ST_S0,1,15.937029,87.578371,MS_S2,M,0.0,0.0,10.0,...,0,0,6,0,0,0,0,0,0,0
3,5/16/2014,ST_S1,1,15.934292,68.438056,MS_S2,M,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
4,5/20/2014,ST_S0,1,15.501711,80.514716,MS_S0,F,0.0,0.0,1.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130081,5/17/2014,ST_S16,NONE,0.131417,46.258727,MS_S2,M,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
130082,5/19/2014,ST_S41,NONE,0.117728,36.574949,MS_S1,M,0.0,0.0,0.0,...,0,0,2,0,0,0,0,0,0,0
130083,5/16/2014,ST_S14,NONE,0.131417,56.906229,MS_S3,M,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
130084,5/15/2014,ST_S14,NONE,0.087611,23.857632,MS_S2,M,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
for 

numpy.float64

In [13]:
data.isna().sum()

DATE_FOR                   0
RTD_ST_CD                  0
CustomerSegment            0
Tenure                     0
Age                        0
MART_STATUS                0
GENDER                     0
CHANNEL1_6M              809
CHANNEL2_6M              809
CHANNEL3_6M              809
CHANNEL4_6M              809
CHANNEL5_6M              809
METHOD1_6M               809
RECENT_PAYMENT           809
PAYMENTS_6M              809
CHANNEL1_3M                0
CHANNEL2_3M                0
CHANNEL3_3M                0
CHANNEL4_3M                0
CHANNEL5_3M                0
METHOD1_3M                 0
PAYMENTS_3M                0
NOT_DI_3M                  0
NOT_DI_6M                  0
EVENT1_30_FLAG             0
EVENT2_90_SUM              0
LOGINS                     0
POLICYPURCHASECHANNEL      0
Call_Flag                  0
dtype: int64

In [17]:
na_cols = ['CHANNEL1_6M', 'CHANNEL2_6M', 'CHANNEL3_6M',
          'CHANNEL4_6M', 'CHANNEL5_6M', 'METHOD1_6M',
          'RECENT_PAYMENT', 'PAYMENTS_6M']

In [7]:
data[data['CHANNEL1_6M'].isna()]

,DATE_FOR,RTD_ST_CD,CustomerSegment,Tenure,Age,MART_STATUS,GENDER,CHANNEL1_6M,CHANNEL2_6M,CHANNEL3_6M,...,CHANNEL5_3M,METHOD1_3M,PAYMENTS_3M,NOT_DI_3M,NOT_DI_6M,EVENT1_30_FLAG,EVENT2_90_SUM,LOGINS,POLICYPURCHASECHANNEL,Call_Flag
365,5/14/2014,ST_S0,1,17.166324,50.609172,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
869,5/15/2014,ST_S20,1,16.098563,43.592060,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1051,5/14/2014,ST_S17,1,16.021903,66.228611,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
1413,5/18/2014,ST_S22,1,38.844627,65.990418,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1468,5/17/2014,ST_S6,1,15.676934,58.392882,MS_S2,F,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129577,5/19/2014,ST_S36,1,0.090349,42.220397,MS_S2,F,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
129688,5/20/2014,ST_S4,NONE,0.087611,50.299795,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
129767,5/20/2014,ST_S27,1,0.087611,35.184120,MS_S1,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
129980,5/19/2014,ST_S38,NONE,0.068446,28.465435,MS_S2,M,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [37]:
for idx, row in data.iterrows():
    for col in na_cols:
        if math.isnan(float(row[col])):
            median = data[col].median()
            data[col].fillna(median, inplace=True)

In [38]:
data.isna().sum()

DATE_FOR                 0
RTD_ST_CD                0
CustomerSegment          0
Tenure                   0
Age                      0
MART_STATUS              0
GENDER                   0
CHANNEL1_6M              0
CHANNEL2_6M              0
CHANNEL3_6M              0
CHANNEL4_6M              0
CHANNEL5_6M              0
METHOD1_6M               0
RECENT_PAYMENT           0
PAYMENTS_6M              0
CHANNEL1_3M              0
CHANNEL2_3M              0
CHANNEL3_3M              0
CHANNEL4_3M              0
CHANNEL5_3M              0
METHOD1_3M               0
PAYMENTS_3M              0
NOT_DI_3M                0
NOT_DI_6M                0
EVENT1_30_FLAG           0
EVENT2_90_SUM            0
LOGINS                   0
POLICYPURCHASECHANNEL    0
Call_Flag                0
dtype: int64

In [30]:
# remove target variable
data = data.drop(['Call_Flag'], axis=1)
data

,DATE_FOR,RTD_ST_CD,CustomerSegment,Tenure,Age,MART_STATUS,GENDER,CHANNEL1_6M,CHANNEL2_6M,CHANNEL3_6M,...,CHANNEL4_3M,CHANNEL5_3M,METHOD1_3M,PAYMENTS_3M,NOT_DI_3M,NOT_DI_6M,EVENT1_30_FLAG,EVENT2_90_SUM,LOGINS,POLICYPURCHASECHANNEL
0,5/19/2014,ST_S0,1,16.175222,78.403833,MS_S0,F,0.0,0.0,1.0,...,0,2,0,3,0,0,0,0,0,0
1,5/17/2014,ST_S0,1,15.931554,70.989733,MS_S1,F,0.0,6.0,0.0,...,0,0,3,3,0,0,0,0,0,0
2,5/15/2014,ST_S0,1,15.937029,87.578371,MS_S2,M,0.0,0.0,10.0,...,0,0,0,6,0,0,0,0,0,0
3,5/16/2014,ST_S1,1,15.934292,68.438056,MS_S2,M,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
4,5/20/2014,ST_S0,1,15.501711,80.514716,MS_S0,F,0.0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130081,5/17/2014,ST_S16,NONE,0.131417,46.258727,MS_S2,M,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
130082,5/19/2014,ST_S41,NONE,0.117728,36.574949,MS_S1,M,0.0,0.0,0.0,...,0,0,0,2,0,0,0,0,0,0
130083,5/16/2014,ST_S14,NONE,0.131417,56.906229,MS_S3,M,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
130084,5/15/2014,ST_S14,NONE,0.087611,23.857632,MS_S2,M,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
col_names = data.columns
print(col_names)

Index(['DATE_FOR', 'RTD_ST_CD', 'CustomerSegment', 'Tenure', 'Age',
       'MART_STATUS', 'GENDER', 'CHANNEL1_6M', 'CHANNEL2_6M', 'CHANNEL3_6M',
       'CHANNEL4_6M', 'CHANNEL5_6M', 'METHOD1_6M', 'RECENT_PAYMENT',
       'PAYMENTS_6M', 'CHANNEL1_3M', 'CHANNEL2_3M', 'CHANNEL3_3M',
       'CHANNEL4_3M', 'CHANNEL5_3M', 'METHOD1_3M', 'PAYMENTS_3M', 'NOT_DI_3M',
       'NOT_DI_6M', 'EVENT1_30_FLAG', 'EVENT2_90_SUM', 'LOGINS',
       'POLICYPURCHASECHANNEL'],
      dtype='object')


In [32]:
# one-hot encoding for categorical variables
categorical_vars = [var for var in data.columns if data[var].dtype == 'O']
len(categorical_vars)
#data = pd.get_dummies(data, columns=categorical_vars)

5

In [35]:
for var in categorical_vars:
    print(data[var].value_counts())
    print(data[var].value_counts() / float(len(data)))
    print(var, " contains ", len(data[var].unique()), " labels.")

5/18/2014    16884
5/14/2014    16787
5/19/2014    16597
5/15/2014    16522
5/16/2014    16476
5/17/2014    15938
5/13/2014    15162
5/20/2014    14911
Name: DATE_FOR, dtype: int64
5/18/2014    0.130603
5/14/2014    0.129853
5/19/2014    0.128383
5/15/2014    0.127803
5/16/2014    0.127447
5/17/2014    0.123286
5/13/2014    0.117283
5/20/2014    0.115341
Name: DATE_FOR, dtype: float64
DATE_FOR  contains  8  labels.
ST_S7     24297
ST_S9     18746
ST_S0      8477
ST_S14     7656
ST_S4      6287
ST_S42     5880
ST_S22     5630
ST_S6      4779
ST_S11     4222
ST_S3      3105
ST_S16     3020
ST_S25     2625
ST_S20     2421
ST_S33     2300
ST_S12     2252
ST_S2      1908
ST_S39     1849
ST_S32     1784
ST_S24     1655
ST_S38     1638
ST_S21     1312
ST_S10     1154
ST_S1      1149
ST_S8      1038
ST_S44     1033
ST_S17      940
ST_S13      933
ST_S50      845
ST_S19      711
ST_S28      709
ST_S40      668
ST_S15      609
ST_S35      597
ST_S41      589
ST_S48      575
ST_S27      545
ST_S1

In [37]:
numerical_vars = [var for var in data.columns if data[var].dtype != 'O']
len(numerical_vars)
numerical_vars

['Tenure',
 'Age',
 'CHANNEL1_6M',
 'CHANNEL2_6M',
 'CHANNEL3_6M',
 'CHANNEL4_6M',
 'CHANNEL5_6M',
 'METHOD1_6M',
 'RECENT_PAYMENT',
 'PAYMENTS_6M',
 'CHANNEL1_3M',
 'CHANNEL2_3M',
 'CHANNEL3_3M',
 'CHANNEL4_3M',
 'CHANNEL5_3M',
 'METHOD1_3M',
 'PAYMENTS_3M',
 'NOT_DI_3M',
 'NOT_DI_6M',
 'EVENT1_30_FLAG',
 'EVENT2_90_SUM',
 'LOGINS',
 'POLICYPURCHASECHANNEL']

In [38]:
print(data[numerical_vars].isnull().sum())

Tenure                   0
Age                      0
CHANNEL1_6M              0
CHANNEL2_6M              0
CHANNEL3_6M              0
CHANNEL4_6M              0
CHANNEL5_6M              0
METHOD1_6M               0
RECENT_PAYMENT           0
PAYMENTS_6M              0
CHANNEL1_3M              0
CHANNEL2_3M              0
CHANNEL3_3M              0
CHANNEL4_3M              0
CHANNEL5_3M              0
METHOD1_3M               0
PAYMENTS_3M              0
NOT_DI_3M                0
NOT_DI_6M                0
EVENT1_30_FLAG           0
EVENT2_90_SUM            0
LOGINS                   0
POLICYPURCHASECHANNEL    0
dtype: int64


In [40]:
print(round(data[numerical_vars].describe()), 2)

         Tenure       Age  CHANNEL1_6M  CHANNEL2_6M  CHANNEL3_6M  CHANNEL4_6M  \
count  129277.0  129277.0     129277.0     129277.0     129277.0     129277.0   
mean        9.0      49.0          0.0          1.0          1.0          0.0   
std         7.0      14.0          1.0          2.0          2.0          1.0   
min         0.0      17.0          0.0          0.0          0.0          0.0   
25%         3.0      38.0          0.0          0.0          0.0          0.0   
50%         7.0      49.0          0.0          0.0          0.0          0.0   
75%        12.0      59.0          0.0          1.0          0.0          0.0   
max        63.0      99.0         12.0         53.0         26.0         18.0   

       CHANNEL5_6M  METHOD1_6M  RECENT_PAYMENT  PAYMENTS_6M  ...  CHANNEL4_3M  \
count     129277.0    129277.0        129277.0     129277.0  ...     129277.0   
mean           1.0         2.0             0.0          5.0  ...          0.0   
std            2.0         

In [5]:
# identify target variable
X = data.drop(columns=['Call_Flag'])
y = data['Call_Flag']

In [6]:
# StandardScaler normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)